# Index

In [35]:
import re
import string
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer

**Membaca crawl_wisata.csv**

In [36]:
df = pd.read_csv('crawl_wisata.csv')
df.head()

,no,title,content,clean_content,url
0,1,PURI KAYANA CAMPING GROUND,kangen ngecamp nggak nih? Cuss berkemah denga...,kangen ngecamp nggak nih cuss berkemah dengan...,https://visitjawatengah.jatengprov.go.id/id/re...
1,2,MENGUNJUNGI WISATA RELIGI NYATNYONO,Desa Nyatnyono di lereng Gunung Ungaran dikena...,desa nyatnyono di lereng gunung ungaran dikena...,https://visitjawatengah.jatengprov.go.id/id/re...
2,3,"CANDI DUKUH, PETILASAN PRABU BRAWIJAYA","Candi Dukuh terletak di Desa Rowoboni, Kecamat...",candi dukuh terletak di desa rowoboni kecamata...,https://visitjawatengah.jatengprov.go.id/id/re...
3,4,5 HAL SPESIAL DI DESA WISATA LEREP,Jalan-jalan ke Semarang? Mampirlah ke Desa Wis...,jalan jalan ke semarang mampirlah ke desa wisa...,https://visitjawatengah.jatengprov.go.id/id/re...
4,5,ALL IN ONE IN SUNRISE HILL GEDONG SONGO,lagi cari referensi resto? Penginapan? Spot s...,lagi cari referensi resto penginapan spot sel...,https://visitjawatengah.jatengprov.go.id/id/re...


**Menggabungkan title dan clean_content dalam 1 kolom**

In [37]:
# df['combined']=df['title']+' '+df['clean_content']

**Menampilkan kata-kata yang ada, lokasi kata, dan jumlah kata pada tiap dokumen, menggunakan *CountVectoriser***

In [21]:
# Activate CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer

# Count Vectorizer
vect = CountVectorizer()  
vects = vect.fit_transform(df['clean_content'])

td = pd.DataFrame(vects.toarray())
td.columns = vect.get_feature_names_out()
term_document_matrix = td.T
term_document_matrix.columns =  ['Doc'+str(i+1) for i in range(td.shape[0])]
term_document_matrix['total_count'] = term_document_matrix.sum(axis=1)

print(term_document_matrix.head())


term_document_matrix.to_csv('term_document.csv')


           Doc1  Doc2  Doc3  Doc4  Doc5  Doc6  Doc7  Doc8  Doc9  Doc10  ...  \
abad          0     0     1     0     0     0     0     0     0      0  ...   
acara         0     0     0     0     0     0     0     0     0      0  ...   
ada           0     0     0     0     1     0     1     0     0      0  ...   
adalah        0     0     0     0     0     0     0     0     0      0  ...   
adrenalin     0     0     0     0     0     0     0     0     0      1  ...   

           Doc30  Doc31  Doc32  Doc33  Doc34  Doc35  Doc36  Doc37  Doc38  \
abad           1      0      0      0      0      0      0      0      0   
acara          0      2      0      0      0      0      0      0      0   
ada            0      0      0      0      0      0      2      1      0   
adalah         0      1      0      0      1      0      1      0      1   
adrenalin      0      0      0      0      0      0      0      0      0   

           total_count  
abad                 2  
acara             

**Tokenization** memecah kata pada hasil kombinasi title dan clean_content yang ada menggunakan

In [38]:
df_parse = df.copy()

cols = ["clean_content"]
for col in cols:
  df_parse[col] = df_parse[col].str.split()

df_parse.head()

,no,title,content,clean_content,url
0,1,PURI KAYANA CAMPING GROUND,kangen ngecamp nggak nih? Cuss berkemah denga...,"[kangen, ngecamp, nggak, nih, cuss, berkemah, ...",https://visitjawatengah.jatengprov.go.id/id/re...
1,2,MENGUNJUNGI WISATA RELIGI NYATNYONO,Desa Nyatnyono di lereng Gunung Ungaran dikena...,"[desa, nyatnyono, di, lereng, gunung, ungaran,...",https://visitjawatengah.jatengprov.go.id/id/re...
2,3,"CANDI DUKUH, PETILASAN PRABU BRAWIJAYA","Candi Dukuh terletak di Desa Rowoboni, Kecamat...","[candi, dukuh, terletak, di, desa, rowoboni, k...",https://visitjawatengah.jatengprov.go.id/id/re...
3,4,5 HAL SPESIAL DI DESA WISATA LEREP,Jalan-jalan ke Semarang? Mampirlah ke Desa Wis...,"[jalan, jalan, ke, semarang, mampirlah, ke, de...",https://visitjawatengah.jatengprov.go.id/id/re...
4,5,ALL IN ONE IN SUNRISE HILL GEDONG SONGO,lagi cari referensi resto? Penginapan? Spot s...,"[lagi, cari, referensi, resto, penginapan, spo...",https://visitjawatengah.jatengprov.go.id/id/re...


In [39]:
# Memasukkan tokenisasi ke dalam df_doc
df_doc = pd.DataFrame(columns = ["document"])
df_doc["document"] = df_parse['clean_content']

df_doc.head()

,document
0,"[kangen, ngecamp, nggak, nih, cuss, berkemah, ..."
1,"[desa, nyatnyono, di, lereng, gunung, ungaran,..."
2,"[candi, dukuh, terletak, di, desa, rowoboni, k..."
3,"[jalan, jalan, ke, semarang, mampirlah, ke, de..."
4,"[lagi, cari, referensi, resto, penginapan, spo..."


**Menambahkan nilai ID dokumen dari lokasi kata ditemukan berdasarkan letak kata pada dokumen ke-n, n=index**

In [40]:
df_term = pd.DataFrame(columns = ["Terms","DocID"])

size = df_doc.shape[0]
for index in range(size):
    row = df_doc['document'].iloc[index]

    for item in row:
        df_term = df_term.append({"Terms":item,"DocID":index}, ignore_index=True)

C:\Users\User\AppData\Local\Temp\ipykernel_2548\2190966126.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_term = df_term.append({"Terms":item,"DocID":index}, ignore_index=True)
C:\Users\User\AppData\Local\Temp\ipykernel_2548\2190966126.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_term = df_term.append({"Terms":item,"DocID":index}, ignore_index=True)
C:\Users\User\AppData\Local\Temp\ipykernel_2548\2190966126.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_term = df_term.append({"Terms":item,"DocID":index}, ignore_index=True)
C:\Users\User\AppData\Local\Temp\ipykernel_2548\2190966126.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pan

In [41]:
df_term

,Terms,DocID
0,kangen,0
1,ngecamp,0
2,nggak,0
3,nih,0
4,cuss,0
...,...,...
2679,dicampur,37
2680,dengan,37
2681,gula,37
2682,aren,37


In [42]:
df_term.shape

(2684, 2)

**Mengurutkan Term berdasar alfabetnya**

In [43]:
df_term_sorted = df_term.sort_values(by="Terms")
df_term_sorted.head(20)

,Terms,DocID
144,abad,2
2044,abad,29
2164,acara,30
2177,acara,30
2512,ada,35
321,ada,6
542,ada,10
767,ada,14
251,ada,4
2498,ada,35


In [44]:
df_term_sorted.shape

(2684, 2)

# inverted index

**Inisialisasi inverted index**

In [45]:
df_invertedindex = pd.DataFrame(columns = ["Terms","docFreq","postList"])
df_invertedindex

,Terms,docFreq,postList


In [46]:
# Memasukkan daftar term
size = df_term_sorted.shape[0]
df_invertedindex['Terms'] = df_term_sorted['Terms'].unique()

df_invertedindex.head()

,Terms,docFreq,postList
0,abad,NaN,NaN
1,acara,NaN,NaN
2,ada,NaN,NaN
3,adalah,NaN,NaN
4,adrenalin,NaN,NaN


In [47]:
# inisialisasi nilai
df_invertedindex["docFreq"]=0
df_invertedindex.head()

,Terms,docFreq,postList
0,abad,0,NaN
1,acara,0,NaN
2,ada,0,NaN
3,adalah,0,NaN
4,adrenalin,0,NaN


In [48]:
size = df_term_sorted.shape[0]
invertedIndexCounter = 0
temp_list = []
for i in range(size):
    if (df_term_sorted['Terms'].iloc[i] == df_invertedindex['Terms'].iloc[invertedIndexCounter]):
        df_invertedindex["docFreq"].iloc[invertedIndexCounter]+=1
        temp_list.append(df_term_sorted["DocID"].iloc[i])
        df_invertedindex["postList"].iloc[invertedIndexCounter] = temp_list
    else :
        temp_list = []
        invertedIndexCounter += 1
        df_invertedindex["docFreq"].iloc[invertedIndexCounter] += 1
        temp_list.append(df_term_sorted["DocID"].iloc[i])
        df_invertedindex["postList"].iloc[invertedIndexCounter] = temp_list


C:\Users\User\AppData\Local\Temp\ipykernel_2548\3755108842.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_invertedindex["docFreq"].iloc[invertedIndexCounter]+=1
C:\Users\User\AppData\Local\Temp\ipykernel_2548\3755108842.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_invertedindex["postList"].iloc[invertedIndexCounter] = temp_list
C:\Users\User\AppData\Local\Temp\ipykernel_2548\3755108842.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-

In [49]:
print(df_invertedindex.head())

       Terms  docFreq                            postList
0       abad        2                             [2, 29]
1      acara        2                            [30, 30]
2        ada        9  [35, 6, 10, 14, 4, 35, 28, 11, 36]
3     adalah        7        [33, 30, 15, 35, 27, 18, 37]
4  adrenalin        2                             [9, 17]


In [50]:
# Export to CSV
print(df_invertedindex)
df_invertedindex.to_csv("invertedIndex.csv",index=False)

           Terms  docFreq                            postList
0           abad        2                             [2, 29]
1          acara        2                            [30, 30]
2            ada        9  [35, 6, 10, 14, 4, 35, 28, 11, 36]
3         adalah        7        [33, 30, 15, 35, 27, 18, 37]
4      adrenalin        2                             [9, 17]
...          ...      ...                                 ...
1046    youghurt        1                                [36]
1047         yuk        3                           [3, 0, 6]
1048      ziarah        1                                [22]
1049  zipcoaster        1                                [10]
1050        zona        1                                [17]

[1051 rows x 3 columns]
